In [1]:
import pandas as pd
from pandas import json_normalize
from textblob import TextBlob


In [3]:
#Se abre y visualiza el archivo
df_games = pd.read_json('steam_games.json', lines=True)
df_games.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [4]:
# Se eliminan las columnas que no serán tenidas en cuenta para el modelamiento
columnas_a_eliminar = ['publisher', 'title', 'url', 'reviews_url', 'specs', 'early_access', 'developer', 'tags']
df_games.drop(columnas_a_eliminar, axis=1, inplace=True)

In [5]:
#Se eliminan los valores nulos debido a que hay muchas filas con valores nulos en todas las columnas
df_games.dropna(inplace=True)


In [6]:
# Se pasa a tipo datetime la columna de release date
df_games["release_date"] = pd.to_datetime(df_games["release_date"], errors='coerce')
# Se rellenan los valores faltantes en las columna 'release_date' con la moda
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    df_games[col].fillna(df_games[col].mode()[0], inplace=True)

In [7]:
# se crea una columna de relase year solo con los años y se elimina release date
df_games['release_year'] = df_games['release_date'].dt.year.astype('Int64')
df_games.drop('release_date', axis=1, inplace=True)

In [8]:
#Se reinicia el indice del dataframe
df_games.reset_index(drop=True, inplace=True)

In [9]:
#Se llenan los nulos de la columna generes con corchetes para poder transformar a listas
df_games['genres'] = df_games['genres'].fillna('[]')

In [10]:
#se transforma la columna genero a listas, se la desanida y utilizamos get dummies para darle un valor numerico
genres_df = pd.DataFrame(df_games['genres'].tolist())
genres_df_obj = genres_df.stack()
genres_df1 = pd.get_dummies(genres_df_obj)
genres_df1 = genres_df1.groupby(level= [0], axis= 0).sum()

In [11]:
#Se evalua el aporte que tiene cada genero para analizar con cual quedarnos
genres_df1.sum().sort_values(ascending=False)/len(genres_df1)

Indie                        0.548104
Action                       0.388058
Casual                       0.292175
Adventure                    0.283159
Strategy                     0.242894
Simulation                   0.235978
RPG                          0.185791
Free to Play                 0.068327
Early Access                 0.050223
Sports                       0.043777
Racing                       0.038056
Massively Multiplayer        0.036354
Design &amp; Illustration    0.016331
Utilities                    0.011659
Web Publishing               0.009451
Animation &amp; Modeling     0.006228
Education                    0.004418
Video Production             0.003838
Software Training            0.003693
Audio Production             0.003114
Photo Editing                0.002788
Accounting                   0.000253
dtype: float64

In [12]:
#Se seleccionan las columnas que me voy a quedar
genres = genres_df1[['Indie','Action','Casual','Adventure','Strategy','Simulation','RPG','Free to Play','Early Access' ,'Sports','Massively Multiplayer','Racing'  ,'Design &amp; Illustration' ,'Utilities']]

In [13]:
#Se unen los dos dataframe y se elimina la columna de genres
df_games = pd.concat([df_games, genres],axis=1)
df_games.drop(columns=['genres'],inplace=True)
df_games.dropna(subset='Indie',inplace=True)

In [14]:
# Se aplica una mascara a la columna price para transforar los valores no numericos a 0
mask = df_games['price'].str.contains(r'free|to play', case=False, na=False)
df_games.loc[mask, 'price'] = 0

In [15]:
# Se cambia la columna 'id' a tipo int
df_games['id'] = df_games['id'].astype('Int64')

In [16]:
# Se lee y visualiza el segundo archivo
with open('users_items.json', "r", encoding='utf-8') as f:
    data = f.readlines()
    
# Convertir la lineas a registros JSON
records = [eval(line.strip()) for line in data]
df_items = pd.DataFrame(records)

df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [17]:
# Se desanida la columna items con la funcion lambda
df_items['playtime_forever'] = df_items['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
df_items['playtime_2weeks'] = df_items['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
df_items['id'] = df_items['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
df_items['id'] = df_items['id'].astype(int)
df_items['playtime_forever'] = df_items['playtime_forever'].astype(int)
df_items['playtime_2weeks'] = df_items['playtime_2weeks'].astype(int)

In [18]:
# Se eliminan las columnas que no serán tenidas en cuenta para el modelamiento
columnas_a_eliminar = ['items_count',	'user_url', 'items']
df_items.drop(columnas_a_eliminar, axis=1, inplace=True)

In [19]:
# Se verifica si hay registros duplicados y se eliminan
hay_duplicados = df_items.duplicated()
df_items = df_items.drop_duplicates()
# Se eliminan los registros totalmente nulos
df_items= df_items.dropna(how='all')

In [20]:
#Se borran los registros de la columna playtime_forever que tengan 0 
df_items = df_items.drop(df_items[df_items['playtime_forever'] == 0].index)

In [21]:
# Se lee y visualiza el tercer archivo
with open('user_reviews.json', "r", encoding='utf-8') as f:
    data = f.readlines()
# Convertir la lineas a registros JSON
records = [eval(line.strip()) for line in data]
df_reviews = pd.DataFrame(records)
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [22]:
# Se desanida la columna reviews
df_reviews= df_reviews.explode('reviews', ignore_index=True)
df_reviews_normalizado= json_normalize(df_reviews['reviews'])

In [23]:
# Se une el df original con el normalizado
df_reviews = df_reviews.join(df_reviews_normalizado)

In [24]:
# Se eliminan las columnas que no serán tenidas en cuenta para el modelamiento
columnas_a_eliminar = ['user_url',	'reviews']
df_reviews.drop(columnas_a_eliminar, axis=1, inplace=True)

In [25]:
# Se verifica si hay registros duplicados y se eliminan
hay_duplicados = df_reviews.duplicated()
df_reviews = df_reviews.drop_duplicates()
# Se eliminan los registros totalmente nulos
df_reviews = df_reviews.dropna(how='all')

In [26]:
# se crea la columna posted year
df_reviews['posted_year'] = df_reviews['posted'].str.extract(r'(\d{4})')
#Se elimina la columna posted
df_reviews.drop('posted' , axis = 1, inplace = True)
#Se eliminan los nulos de year
df_reviews.dropna(subset='posted_year', inplace=True)

In [27]:
#Se crea una funcion para detectar el sentimiento
def analisis_sentimiento(texto):
        analisis = TextBlob(texto)
        #se saca la polaridad
        polaridad = analisis.sentiment.polarity
        if polaridad < 0:
            return 0  # Malo
        elif polaridad > 0:
            return 2  # Positivo
        else:
            return 1  # Neutral

In [28]:
#se crea una columna 'sentiment_analysis'
df_reviews['sentiment_analysis'] = df_reviews['review'].apply(analisis_sentimiento)

In [29]:
# Se unen los dataframe games e items por su id. Y a su vez se une con el df reviews por el user_id
df_games_items = pd.merge(df_games, df_items,on='id')
df_final = pd.merge(df_games_items, df_reviews, on = 'user_id')

In [30]:
#Se guarda el archivo final 
df_final.to_csv('data_final.csv', index=False)

In [32]:
df_final.user_id


0                  kube134
1        76561198030567998
2        76561198030567998
3        76561198030567998
4        76561198030567998
               ...        
38149    76561198085345522
38150    76561198085345522
38151       Shadowbolt4231
38152    76561198045631910
38153    76561198152281826
Name: user_id, Length: 38154, dtype: object